## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

# Adding numpy for the elimination of blank hotel names
import numpy as np

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
0,0,Bredasdorp,-34.5322,20.0403,60.28,82,89,4.61,ZA,2022-01-29 00:54:08,overcast clouds
1,1,Hobart,-42.8794,147.3294,68.27,78,75,13.80,AU,2022-01-29 00:52:51,broken clouds
2,2,Constitucion,-35.3333,-72.4167,57.36,92,3,5.68,CL,2022-01-29 00:54:08,clear sky
3,3,Punta Arenas,-53.1500,-70.9167,68.11,22,0,8.05,CL,2022-01-29 00:54:09,clear sky
4,4,Gazli,40.1333,63.4500,34.74,75,45,11.68,UZ,2022-01-29 00:54:09,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 55


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
16,16,Liverpool,53.4106,-2.9779,53.29,85,100,3.00,GB,2022-01-29 00:54:13,overcast clouds
95,95,Brownsville,25.9018,-97.4975,53.10,77,100,1.01,US,2022-01-29 00:55:30,overcast clouds
138,138,Alexandria,31.2156,29.9553,51.76,87,40,3.44,EG,2022-01-29 00:56:28,scattered clouds
149,149,Skibbereen,51.5500,-9.2667,51.60,90,100,21.45,IE,2022-01-29 00:57:49,overcast clouds
155,155,Ayr,55.4627,-4.6339,52.32,87,100,19.57,GB,2022-01-29 00:57:51,overcast clouds
230,230,Umm Kaddadah,13.6017,26.6876,53.65,19,91,18.12,SD,2022-01-29 00:59:11,overcast clouds
295,295,Agadez,19.7500,10.2500,54.03,19,0,9.55,NE,2022-01-29 01:01:35,clear sky
301,301,Jalu,29.0331,21.5482,53.22,74,90,6.85,LY,2022-01-29 01:01:36,overcast clouds
326,326,Saint George,37.1041,-113.5841,54.07,27,0,2.98,US,2022-01-29 01:00:10,clear sky
343,343,Nawabganj,24.5911,88.2710,53.26,68,0,7.49,BD,2022-01-29 01:02:48,clear sky


In [5]:
# 4a. Determine if there are any empty rows.

preferred_cities_df.isnull().sum()



City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Country                0
Date                   0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
#assumption is if any of the 11 columns are NA we removed the entire row of data.

clean_df= preferred_cities_df.dropna(thresh=11)

clean_df.isnull().sum()

City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Country                0
Date                   0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
16,Liverpool,GB,53.29,overcast clouds,53.4106,-2.9779,
95,Brownsville,US,53.10,overcast clouds,25.9018,-97.4975,
138,Alexandria,EG,51.76,scattered clouds,31.2156,29.9553,
149,Skibbereen,IE,51.60,overcast clouds,51.5500,-9.2667,
155,Ayr,GB,52.32,overcast clouds,55.4627,-4.6339,
230,Umm Kaddadah,SD,53.65,overcast clouds,13.6017,26.6876,
295,Agadez,NE,54.03,clear sky,19.7500,10.2500,
301,Jalu,LY,53.22,overcast clouds,29.0331,21.5482,
326,Saint George,US,54.07,clear sky,37.1041,-113.5841,
343,Nawabganj,BD,53.26,clear sky,24.5911,88.2710,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [9]:

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():
      # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.-see below

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search-see below
    
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city-see below
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    
hotel_df   

Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
16,Liverpool,GB,53.29,overcast clouds,53.4106,-2.9779,Premier Inn Liverpool City Centre (Moorfields)...
95,Brownsville,US,53.10,overcast clouds,25.9018,-97.4975,University Inn & Suites
138,Alexandria,EG,51.76,scattered clouds,31.2156,29.9553,Royal Crown Hotel
149,Skibbereen,IE,51.60,overcast clouds,51.5500,-9.2667,The West Cork Hotel
155,Ayr,GB,52.32,overcast clouds,55.4627,-4.6339,Mercure Ayr Hotel
230,Umm Kaddadah,SD,53.65,overcast clouds,13.6017,26.6876,
295,Agadez,NE,54.03,clear sky,19.7500,10.2500,
301,Jalu,LY,53.22,overcast clouds,29.0331,21.5482,صاحبة الفخامة
326,Saint George,US,54.07,clear sky,37.1041,-113.5841,Best Western Plus Abbey Inn
343,Nawabganj,BD,53.26,clear sky,24.5911,88.2710,দা সফ্‌ট আইটি


In [10]:
hotel_df.isnull().sum()

#  Strange that the blanks in the Hotel Name column are not showing up as null?  Maybe need to convert to NaN?

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [11]:
# 7. Drop the rows where there is no Hotel Name.
#  First include conversion of blanks to NaN as noted above to mark blanks so can then use dropna()
#  Used similar method to Deliverable 1 in PyCitySchools_Challenge:  student_data_df.loc[(student_data_df['grade'] =='9th') & (student_data_df['school_name']=='Thomas High School'), 'reading_score']=np.NaN

hotel_df.loc[(hotel_df['Hotel Name'] ==''), 'Hotel Name']=np.NaN

clean_df= hotel_df.dropna()

clean_df


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
16,Liverpool,GB,53.29,overcast clouds,53.4106,-2.9779,Premier Inn Liverpool City Centre (Moorfields)...
95,Brownsville,US,53.10,overcast clouds,25.9018,-97.4975,University Inn & Suites
138,Alexandria,EG,51.76,scattered clouds,31.2156,29.9553,Royal Crown Hotel
149,Skibbereen,IE,51.60,overcast clouds,51.5500,-9.2667,The West Cork Hotel
155,Ayr,GB,52.32,overcast clouds,55.4627,-4.6339,Mercure Ayr Hotel
301,Jalu,LY,53.22,overcast clouds,29.0331,21.5482,صاحبة الفخامة
326,Saint George,US,54.07,clear sky,37.1041,-113.5841,Best Western Plus Abbey Inn
343,Nawabganj,BD,53.26,clear sky,24.5911,88.2710,দা সফ্‌ট আইটি
358,Misratah,LY,53.06,clear sky,32.3754,15.0925,Almasa Hotel
362,Ferrol,ES,50.27,clear sky,43.4832,-8.2369,Parador de Ferrol


In [12]:
# 8a. Create the output File (CSV)

output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv

clean_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_df[["Lat", "Lng"]]

hotel_info

['\n<dl>\n<dt>Hotel Name</dt><dd>Premier Inn Liverpool City Centre (Moorfields) hotel</dd>\n<dt>City</dt><dd>Liverpool</dd>\n<dt>Country</dt><dd>GB</dd>\n<dt>Current Weather</dt><dd>overcast clouds</dd>\n<dt>Max Temp</dt><dd>53.29 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>University Inn & Suites</dd>\n<dt>City</dt><dd>Brownsville</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Current Weather</dt><dd>overcast clouds</dd>\n<dt>Max Temp</dt><dd>53.1 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Royal Crown Hotel</dd>\n<dt>City</dt><dd>Alexandria</dd>\n<dt>Country</dt><dd>EG</dd>\n<dt>Current Weather</dt><dd>scattered clouds</dd>\n<dt>Max Temp</dt><dd>51.76 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>The West Cork Hotel</dd>\n<dt>City</dt><dd>Skibbereen</dd>\n<dt>Country</dt><dd>IE</dd>\n<dt>Current Weather</dt><dd>overcast clouds</dd>\n<dt>Max Temp</dt><dd>51.6 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Mercure Ayr Hotel</dd>\n<dt>City</dt><dd>Ayr</dd>\n<dt>Countr

In [14]:
#make heatmap--didnt specifically list out this step in the directions

# Add a heatmap of temperature for the vacation spots.
locations = clean_df[["Lat", "Lng"]]
max_temp = clean_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig



Figure(layout=FigureLayout(height='420px'))

In [15]:
# 11a. Add a marker layer for each city to the map. 

locations = clean_df[["Lat", "Lng"]]
max_temp = clean_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


In [16]:
# 11b. Display the figure

fig

Figure(layout=FigureLayout(height='420px'))